# Limpieza y Transformación de Datos (ETL)

### 1. Objetivo de la etapa
En esta etapa se realizan las transformaciones necesarias para preparar los datos
para el modelado, a partir de los hallazgos del EDA.


## 2. Importacion de librerias y carga de datos

In [1]:
import pandas as pd
import numpy as np
 
df = pd.read_csv('../data/raw/listings.csv')

C:\Users\brian\AppData\Local\Temp\ipykernel_6536\1356392705.py:4: DtypeWarning: Columns (61,62,95) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/listings.csv')


## 3. Limpieza de la variable objetivo (price)

### 3.1 Conversión de price a numérico


In [2]:
df['price'] = (df['price']
               .str.replace('$', '',regex=False)
               .str.replace(',', '',regex=False)
               .astype(float)
)

### 3.2 Validación post-limpieza

In [3]:
df['price'].dtype

dtype('float64')

In [4]:
df['price'].isna().sum()

np.int64(0)

In [5]:
df['price'].describe()

count     23729.000000
mean       4014.875595
std       16075.326378
min           0.000000
25%        1394.000000
50%        2124.000000
75%        3319.000000
max      663732.000000
Name: price, dtype: float64

## 4. Tratamiento de outliers

### 4.1 Definicion del umbral para outliers

In [6]:
per_99 = df['price'].quantile(0.99) #Se utiliza el percentil 99 para eliminar outliers, a partir del EDA.
per_99  

np.float64(30442.040000000386)

### 4.2 Filtrado de valores extremos

In [7]:
df.shape

(23729, 106)

In [8]:
df = df[df['price'] <= per_99]

df.shape

(23491, 106)

## 5. Seleccion inicial de variables

In [9]:
cols = ['price',
        'accommodates',
        'bedrooms',
        'bathrooms',
        'room_type',
        'neighbourhood_cleansed',
        'number_of_reviews',
        'reviews_per_month'
        ]

df = df[cols]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23491 entries, 0 to 23728
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   price                   23491 non-null  float64
 1   accommodates            23491 non-null  int64  
 2   bedrooms                23448 non-null  float64
 3   bathrooms               23429 non-null  float64
 4   room_type               23491 non-null  object 
 5   neighbourhood_cleansed  23491 non-null  object 
 6   number_of_reviews       23491 non-null  int64  
 7   reviews_per_month       17142 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.6+ MB


## 6. Tratamiento de valores nulos y tipos de datos

In [11]:
df.isna().sum() #Se visualizan los valores nulos, se decide eliminar las filas con valores nulos.

price                        0
accommodates                 0
bedrooms                    43
bathrooms                   62
room_type                    0
neighbourhood_cleansed       0
number_of_reviews            0
reviews_per_month         6349
dtype: int64

### 6.1 Tratamiento de nulos

In [12]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].median())
df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].median())

In [13]:
df.isna().sum()

price                     0
accommodates              0
bedrooms                  0
bathrooms                 0
room_type                 0
neighbourhood_cleansed    0
number_of_reviews         0
reviews_per_month         0
dtype: int64

### 6.2 Correcion de tipos de datos

In [14]:
df['bedrooms'] = df['bedrooms'].round().astype(int) 
df['bathrooms'] = df['bathrooms'].round().astype(int)

In [15]:
df.dtypes

price                     float64
accommodates                int64
bedrooms                    int64
bathrooms                   int64
room_type                  object
neighbourhood_cleansed     object
number_of_reviews           int64
reviews_per_month         float64
dtype: object

## 7. Econding de variables categoricas

### 7.1 One-Hot Encoding

In [16]:
df_encoded = pd.get_dummies(
             df,
             columns=['room_type', 'neighbourhood_cleansed'],
             drop_first=True   
)

In [17]:
df_encoded.shape
df_encoded.head()

,price,accommodates,bedrooms,bathrooms,number_of_reviews,reviews_per_month,room_type_Hotel room,room_type_Private room,room_type_Shared room,neighbourhood_cleansed_Almagro,...,neighbourhood_cleansed_Villa Gral. Mitre,neighbourhood_cleansed_Villa Lugano,neighbourhood_cleansed_Villa Luro,neighbourhood_cleansed_Villa Ortuzar,neighbourhood_cleansed_Villa Pueyrredon,neighbourhood_cleansed_Villa Real,neighbourhood_cleansed_Villa Riachuelo,neighbourhood_cleansed_Villa Santa Rita,neighbourhood_cleansed_Villa Soldati,neighbourhood_cleansed_Villa Urquiza
0,3983.0,2,1,1,26,0.27,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1593.0,1,1,1,20,0.16,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2987.0,2,1,1,1,0.06,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2987.0,2,1,1,0,0.00,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2987.0,2,1,1,66,1.89,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


## 8. Dataset para modelado

In [18]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23491 entries, 0 to 23728
Data columns (total 56 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   price                                     23491 non-null  float64
 1   accommodates                              23491 non-null  int64  
 2   bedrooms                                  23491 non-null  int64  
 3   bathrooms                                 23491 non-null  int64  
 4   number_of_reviews                         23491 non-null  int64  
 5   reviews_per_month                         23491 non-null  float64
 6   room_type_Hotel room                      23491 non-null  bool   
 7   room_type_Private room                    23491 non-null  bool   
 8   room_type_Shared room                     23491 non-null  bool   
 9   neighbourhood_cleansed_Almagro            23491 non-null  bool   
 10  neighbourhood_cleansed_Balvanera       

In [19]:
df_encoded.to_csv('../data/processed/listings_processed.csv', index=False)